### Run the cell below and update the Mongo Host and Port Info

In [1]:
! bash ~/PySyft/scripts/dev_tools.sh

-----------------------------
Containers | Exposed Ports
-----------------------------
/canada-seaweedfs-1  49159 49158 
/canada-queue-1  49153 
/canada-mongo-1  49156 
/canada-redis-1  49155 
/canada-jaeger-1  14268 49154 


In [2]:
## Start here.

In [1]:
# Set your public ports for mongo here.

class EnvSettings:
    MONGO_HOST = "localhost"  # # change to 'localhost'
    MONGO_PORT = "49156"  # change to public port
    MONGO_USERNAME = "root"
    MONGO_PASSWORD = "example"  # nosec
    
env_settings = EnvSettings()

In [2]:
import syft as sy
from syft.core.node.new.user import UserCreate, User, UserUpdate
from syft.core.node.new.mongo_document_store import MongoCollection, MongoDocumentStore, MongoBsonObject
from syft.core.node.new.document_store import ClientConfig
from syft.core.node.common.node_table.syft_object import SyftObjectRegistry
from syft.core.node.worker import Worker
from syft.core.node.new.user_stash import UserStash
from syft.core.common.uid import UID
from syft.core.node.new.document_store import UIDPartitionKey

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/shubham/PySyft


In [3]:
mongo_client_config = ClientConfig(
    hostname=env_settings.MONGO_HOST,
    port=env_settings.MONGO_PORT,
    username=env_settings.MONGO_USERNAME,
    password=env_settings.MONGO_PASSWORD,
)

In [4]:
store = MongoDocumentStore(client_config=mongo_client_config)

In [5]:
mongo_store_ser = sy.serialize(store, to_bytes=True)

In [6]:
store = sy.deserialize(mongo_store_ser, from_bytes=True)

In [7]:
user_stash = UserStash(store=store)

In [8]:
user_stash.partition.db_collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'User_index_name': {'v': 2,
  'key': [('email', 1), ('signing_key', 1), ('verify_key', 1)],
  'unique': True}}

In [9]:
new_user = UserCreate(
    name="Shubham", email="alice1@openmined.org", password="pass", password_verify="pass"
)

In [10]:
user = new_user.to(User)

In [11]:
bson_dict = user.to(MongoBsonObject)

In [12]:
u = MongoBsonObject.to_syft_obj(bson_dict, User)

In [13]:
response = user_stash.set(user)

In [14]:
response

Ok(syft.core.node.new.user.User)

In [15]:
# #### Create indexes
# from pymongo import IndexModel, ASCENDING, DESCENDING
# user_stash.partition.db_collection.create_index([
#     ("email", ASCENDING),
#     ("verify_key", ASCENDING)
# ], unique=True)

In [16]:
search_result = user_stash.get_by_email(email="alice1@openmined.org")


In [17]:
search_result.ok() if search_result.is_ok() else search_result.err()

```python
class User:
  id: str = a336c5c519034176afa983ad52a88a2e
  email: str = "alice1@openmined.org"
  name: str = "Shubham"
  hashed_password: str = "$2b$12$FQwpWiRSBRxfLuHL8CmPLehFHFkq5k553P.OcL9lpIEoMNMtpn4K2"
  salt: str = "$2b$12$FQwpWiRSBRxfLuHL8CmPLe"
  signing_key: str = e0c48bd64d49153c4edac30cba07e2a2b05fac326dfd3b1afa5b7e82e37f56f0
  verify_key: str = 3b8431f8f4ffcaffcb14dbb15750decdb1effdc96d95760deb1fbb314c8c0261
  role: str = ServiceRole.GUEST
  institution: str = None
  website: str = None
  created_at: str = None

```

In [18]:
uid_to_update = UID.from_string("a336c5c519034176afa983ad52a88a2e")

In [19]:
update_user = UserUpdate(id=uid_to_update, email="alice1@openmined.org", name="MyBob2", institution="OpenMined")

In [20]:
update_result = user_stash.update(user=update_user.to(User))

In [21]:
update_result.ok() if update_result.is_ok() else update_result.err()

```python
class User:
  id: str = a336c5c519034176afa983ad52a88a2e
  email: str = "alice1@openmined.org"
  name: str = "MyBob2"
  hashed_password: str = None
  salt: str = None
  signing_key: str = None
  verify_key: str = None
  role: str = None
  institution: str = "OpenMined"
  website: str = None
  created_at: str = None

```

In [22]:
user_stash.get_by_email(email="bob@alice.com")

Ok(None)

In [23]:
uid_to_delete = UID.from_string("a336c5c519034176afa983ad52a88a2e")
response = user_stash.delete_by_uid(uid=uid_to_delete)

In [24]:
response

Ok(<class 'syft.core.node.new.response.SyftSuccess'>: ID: a336c5c519034176afa983ad52a88a2e deleted)

### Testing the services

In [9]:
import syft as sy
from syft.core.node.new.user import UserSearch, UserCreate
from syft.core.node.worker import Worker

In [10]:
from syft.core.node.new.document_store import DocumentStore, DictDocumentStore

In [11]:
worker = Worker(env_settings=env_settings, in_memory=False)

Index Created for Dataset with indexes: [('name', 1)]
Starting Worker: Loving Trask - 8523aa1e1bcb4e37a4a7c249278ec1a9 [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>]


In [12]:
worker.document_store

In [13]:
client = sy.new_login(node=worker, email="info@openmined.org", password="changethis", port=8081)

Logged into Loving Trask as <info@openmined.org>


In [14]:
u_create = UserCreate(name="Shubham", email="shubh@openmined.org", password="pass", password_verify="pass")

In [15]:
client.api.services.user.create(user_create=u_create)

```python
class UserView:
  id: str = cd603a39c6c5420193698125330a718e
  email: str = "shubh@openmined.org"
  name: str = "Shubham"
  role: str = ServiceRole.GUEST
  password: str = None
  password_verify: str = None
  verify_key: str = None
  institution: str = None
  website: str = None

```

In [16]:
client.api.services.user.search(name="Shubham")

,type
0,syft.core.node.new.user.UserView
1,syft.core.node.new.user.UserView


In [17]:
uid_to_delete = UID.from_string("cd603a39c6c5420193698125330a718e")
client.api.services.user.delete(uid_to_delete)

<class 'syft.core.node.new.response.SyftSuccess'>: ID: cd603a39c6c5420193698125330a718e deleted

In [18]:
client.api.services.user.search(name="Shubham")

,type
0,syft.core.node.new.user.UserView
